Set up

In [ ]:
# !pip install openai
# from openai import OpenAI
# !pip install langchain
# !pip install langchain-chroma
# !pip install langchain-openai

import os
from google.colab import drive
import pandas as pd
import math
import matplotlib.pyplot as plt

from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
import os
import getpass
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI


from google.colab import drive
drive.mount('/content/drive')


path_personality = "/content/drive/MyDrive/Rochester/ROC HCI/Misinformation/Rank Blocks/Small RAG Sample Dataset.csv"
personality_data = pd.read_csv(path_personality)
print(len(personality_data))

path_rankings = "/content/drive/MyDrive/Rochester/ROC HCI/Misinformation/Rank Blocks/Rankings Sample Dataset.csv"
ranking_data = pd.read_csv(path_rankings)
print(len(ranking_data))

# number of participants we're returning
K = 3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 40.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.0/121.0 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 2.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0

CSV to Documents in DB

NOTE: It seems like you have to wipe the DB because it remembers prior insertions and will cause duplicates to appear. Weird.

In [ ]:
def rank_block_process_database(data):
  # document loading
  # loader = TextLoader(path_personality)
  # loader.load()

  # text splitting
  columns = []
  for item in data:
    columns.append(item)

  documents = []
  for item in range(len(data)):
    # print(data.iloc[item])
    # print("\n")
    temp = ""
    for col in columns:
      temp += str(col) + ": " + str(data[col][item]) + ", "
    # documents.append(str(data[columns].loc[item]))
    documents.append(temp)
  print("check 1")
  for item in documents:
    print(item)
  print("\n\n")
  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      model_name="gpt-3.5",
      chunk_size=100000,
      chunk_overlap=0,
  )
  documents = text_splitter.create_documents(documents)
  print("check 2")
  for doc in documents:
    print(doc)
  print("\n\n")

  # text embedding
  embeddings_model = OpenAIEmbeddings(api_key="HIDDEN")

  # vector store

  # Load the document, split it into chunks, embed each chunk and load it into the vector store.

  print(len(documents))
  db = Chroma.from_documents(documents, embeddings_model)
  db.delete_collection()
  db = Chroma.from_documents(documents, embeddings_model)
  for item in db.get()["documents"]:
    print(item)
  print(len(db.get()["documents"]))
 
  return embeddings_model, db

embeddings_model, db = rank_block_process_database(personality_data.head(9))

check 1
UserID: 1, Age Range: 18-24, Gender: Female, Race: White, Political Affiliation: Democrat, Strength of Political Affiliation: Very strong, Location (State): Wyoming, Highest Educational Level: Graduate Degree, Occupation: Teacher, Is Reserved: 5, Is Generally Trusting: 3, Tends To Be Lazy: 2, Is Relaxed: 5, Has Few Artistic Intersts: 2, Is Outgoing: 1, Tends to Find Faults: 5, Does a Thorough Job: 2, Gets Nervous Easily: 5, Has an Active Imagination: 3, 
UserID: 2, Age Range: 65 or older, Gender: Non Binary, Race: White, Political Affiliation: Republican, Strength of Political Affiliation: Not strong at all, Location (State): Texas, Highest Educational Level: Associate Degree, Occupation: Researcher, Is Reserved: 4, Is Generally Trusting: 4, Tends To Be Lazy: 3, Is Relaxed: 2, Has Few Artistic Intersts: 3, Is Outgoing: 4, Tends to Find Faults: 3, Does a Thorough Job: 3, Gets Nervous Easily: 5, Has an Active Imagination: 2, 
UserID: 3, Age Range: 24 - 35, Gender: Male, Race: Bla

Retrieving Most Similar Queries / Embeddings

In [ ]:
def rank_block_document_retrieval(data_to_find, embeddings_model, db):
  # three options on how to do it, generall return same things.
  # embedded_query = embeddings_model.embed_query(data_to_find)
  # docs = db.similarity_search_by_vector_with_relevance_scores(embedded_query)
  # print(docs[0][0].page_content, docs[0][1])
  # print(docs[1][0].page_content, docs[1][1])
  # print(docs[2][0].page_content, docs[2][1])
  # print("\n")

  # # here, I believe higher is better (query version)
  # # user 10 is user 6; successfully finds 6 as most similar
  # query = data_to_find
  # docs = db.similarity_search_with_relevance_scores(query, k=3)
  # print(docs[0][0].page_content, docs[0][1])
  # print(docs[1][0].page_content, docs[1][1])
  # print(docs[2][0].page_content, docs[2][1])
  # print("\n")

  query = data_to_find
  docs = db.similarity_search_with_relevance_scores(query, k=K)
  print(docs[0][0].page_content, docs[0][1])
  print(docs[1][0].page_content, docs[1][1])
  print(docs[2][0].page_content, docs[2][1])

  ret = {}

  for people in range(K):
    ret[people] = {"persona": docs[people][0].page_content, "similarity score": docs[people][1]}

  return ret

data_to_find = ""
for col in personality_data:
    data_to_find += str(col) + ": " + str(personality_data[col][9]) + ", "
test_person1 = rank_block_document_retrieval(data_to_find, embeddings_model, db)

data_to_find = ""
for col in personality_data:
    data_to_find += str(col) + ": " + str(personality_data[col][10]) + ", "
test_person2 = rank_block_document_retrieval(data_to_find, embeddings_model, db)

test_person3 = rank_block_document_retrieval('UserID: 42, Age Range: 65 or older, Gender: Female, Race: Latino, Political Affiliation: Democrat, Strength of Political Affiliation: Not strong at all, Location (State): Colorado, Highest Educational Level: Bachelor\'s Degree, Occupation: Writer, Is Reserved: 3, Is Generally Trusting: 3, Tends To Be Lazy: 3, Is Relaxed: 3, Has Few Artistic Intersts: 3, Is Outgoing: 3, Tends to Find Faults: 3, Does a Thorough Job: 3, Gets Nervous Easily: 3, Has an Active Imagination: 3, ', embeddings_model, db)


UserID: 6, Age Range: 55 - 65, Gender: Male, Race: White, Political Affiliation: Democrat, Strength of Political Affiliation: Strong, Location (State): California, Highest Educational Level: High School or Equivalent, Occupation: Doctor, Is Reserved: 3, Is Generally Trusting: 4, Tends To Be Lazy: 3, Is Relaxed: 3, Has Few Artistic Intersts: 2, Is Outgoing: 2, Tends to Find Faults: 3, Does a Thorough Job: 1, Gets Nervous Easily: 1, Has an Active Imagination: 5, 0.9471191130850105
UserID: 4, Age Range: 35 - 45, Gender: Male, Race: Asian, Political Affiliation: Republican, Strength of Political Affiliation: Very strong, Location (State): Washington, Highest Educational Level: Bachelor's Degree, Occupation: Driver, Is Reserved: 3, Is Generally Trusting: 1, Tends To Be Lazy: 4, Is Relaxed: 1, Has Few Artistic Intersts: 5, Is Outgoing: 5, Tends to Find Faults: 1, Does a Thorough Job: 5, Gets Nervous Easily: 3, Has an Active Imagination: 1, 0.9428830913918624
UserID: 5, Age Range: 45 - 55, Ge

Based on Top Results, Find Rankings

In [ ]:
# formula for identifying trusted sources
# give each similar person a weight (weight = similarity score, which is on a scale of 0 - 1)
# top rank would be 3 * person weight, next top rank would be 2 * person weight, final rank would be person weight (done for all ranked shown)

def rank_block_trusted_sources(similar_people):

  trusted_sources = {"The New York Times": 0, "CNN": 0,
                     "Wall Street Journal": 0, "AP News": 0,
                     "The Washington Post": 0, "Fox News": 0,
                     "New York Post": 0, "The Blaze": 0,
                     "Chicago Tribune": 0, "The Hill": 0,
                     "Washington Times": 0}

  # should go through in order since they're returned in order of similarity
  for found_participant in similar_people:
    split = similar_people[found_participant]["persona"].split(",")
    space = split[0].find(" ")
    id = split[0][space+1:]
    weight = float(similar_people[found_participant]["similarity score"])

    for player, s1, s2, s3 in zip(ranking_data["UserID"], ranking_data["Top News Source 1"], ranking_data["Top News Source 2"], ranking_data["Top News Source 3"]):
      if str(player) == str(id):
        trusted_sources[s1] += weight * 3
        trusted_sources[s2] += weight * 2
        trusted_sources[s3] += weight

  print("Sources to pass to Debunking Finder Block:")
  sorted_data = sorted(trusted_sources.items(), key=lambda x: x[1], reverse=True)
  print(sorted_data)
  trusted_sources = {}


  # only returning top 3 scores
  # sending all tied with third place weighting
  bottom = 0
  count = 0
  for i in sorted_data:
    if count < 3:
      print(i)
      trusted_sources[i[0]] = i[1]
      bottom = i[1]
    elif count >= 3:
      if bottom == i[1]:
        trusted_sources[i[0]] = i[1]
    count += 1

  print(trusted_sources)

  return trusted_sources

print(test_person1)
print(rank_block_trusted_sources(test_person2))



{0: {'persona': 'UserID: 6, Age Range: 55 - 65, Gender: Male, Race: White, Political Affiliation: Democrat, Strength of Political Affiliation: Strong, Location (State): California, Highest Educational Level: High School or Equivalent, Occupation: Doctor, Is Reserved: 3, Is Generally Trusting: 4, Tends To Be Lazy: 3, Is Relaxed: 3, Has Few Artistic Intersts: 2, Is Outgoing: 2, Tends to Find Faults: 3, Does a Thorough Job: 1, Gets Nervous Easily: 1, Has an Active Imagination: 5,', 'similarity score': 0.9471191130850105}, 1: {'persona': "UserID: 4, Age Range: 35 - 45, Gender: Male, Race: Asian, Political Affiliation: Republican, Strength of Political Affiliation: Very strong, Location (State): Washington, Highest Educational Level: Bachelor's Degree, Occupation: Driver, Is Reserved: 3, Is Generally Trusting: 1, Tends To Be Lazy: 4, Is Relaxed: 1, Has Few Artistic Intersts: 5, Is Outgoing: 5, Tends to Find Faults: 1, Does a Thorough Job: 5, Gets Nervous Easily: 3, Has an Active Imagination